In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
mimiciv_dir = 'C:/Users/suzie/Dropbox (MIT)/Spring 2021/6.871 Machine Learning for Healthcare/mimic-iv-1.0/hosp'

In [3]:
mimiciv_dir = 'data'

## Part I: read data

In [4]:
df_event = pd.read_csv(f'{mimiciv_dir}/diagnoses_icd.csv', low_memory=False)

In [5]:
df_code = pd.read_csv(f'{mimiciv_dir}/d_icd_diagnoses.csv', low_memory=False)

In [6]:

#merge tables:
df_sub = pd.merge(df_event, df_code, on = ['icd_code','icd_version'], how='left')

#convert to lower case:
df_sub['long_title'] = df_sub['long_title'].str.lower()

df_sub = df_sub.dropna(subset=['icd_code','icd_version','hadm_id','subject_id','long_title'], how='any')

df_sub.head(5)

subject_id   hadm_id  seq_num icd_code  icd_version  \
0    15734973  20475282        3     2825            9   
1    15734973  20475282        2    V0251            9   
2    15734973  20475282        5     V270            9   
3    15734973  20475282        1    64891            9   
4    15734973  20475282        4    66481            9   

                                          long_title  
0                                  sickle-cell trait  
1  carrier or suspected carrier of group b strept...  
2               outcome of delivery, single liveborn  
3  other current conditions classifiable elsewher...  
4  other specified trauma to perineum and vulva, ...

## Part II: ICD Code Cohorting

In [7]:
#Filter to heart failure: 
#Filter 1: long title contains "heart failure" AND long title doesnt contain without heart failure
# this paper: https://academic.oup.com/aje/article/183/5/462/2462157?login=true used almost all of these for ADHF
df_f1 = df_sub[df_sub['long_title'].str.contains('heart failure')]
df_f1 = df_f1[~df_f1['long_title'].str.contains('without heart failure')]
# df_f1.head(5)

In [8]:
df_f1_hadm = df_f1[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
print(np.shape(df_f1_hadm)[0])
n1 = np.shape(df_f1_hadm)[0]
# df_f1_hadm.head(5)

64686


In [9]:
# Filter 2: Filter 1 + exclude "XX disease with heart failure" format (only keep HF as main target in title)
df_f2 = df_f1[~df_f1['long_title'].str.contains('disease')]
df_f2_hadm = df_f2[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
print(np.shape(df_f2_hadm)[0])
hf2_icd = df_f2[['long_title', 'icd_code','icd_version']].drop_duplicates(subset=['icd_code']) #.sort_values('icd_code')
hf2_icd
n2 = np.shape(df_f2_hadm)[0]

64566


In [10]:
# Filter 5: Filter 1+2+ Use I50+ and 428+
# source: https://icd.codes/icd10cm/I503 "The ICD code I50 is used to code Acute decompensated heart failure"
df_f5 = df_f2[df_f2['icd_code'].str.contains('428') | df_f2['icd_code'].str.contains('I50')]
df_f5_hadm = df_f5[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
n5=(np.shape(df_f5_hadm)[0])
n5

64409

In [11]:
# Filter 6: Filter 1+2+5+ exclude acute only, exclude chronic only, keep acute on chronic or specified
# source: NA_CARE_Prov_ICD_10_CM_Documentation_and_Coding_Best_Practices_Heart_Failure_v3_ENG_2019 pdf
df_f6 = df_f5[~df_f5['long_title'].str.contains('acute')  | df_f5['long_title'].str.contains('acute on chronic') ]
df_f6 = df_f6[~df_f6['long_title'].str.contains('chronic')  | df_f6['long_title'].str.contains('acute on chronic') ]
df_f6_hadm = df_f6[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
n6 = (np.shape(df_f6_hadm)[0])
n6

51330

In [12]:
# Filter 7: Filter 1+2+5+6 exclude end stage
# It is important to distinguish between AdHF and end-stage HF
# source https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6963179/
df_f7 = df_f6[~df_f6['long_title'].str.contains('end stage')]
df_f7_hadm = df_f7[['subject_id','hadm_id','long_title', 'icd_code','icd_version']].drop_duplicates(subset=['hadm_id'])
n7 = (np.shape(df_f7_hadm)[0])
n7

51303

In [13]:
df_f7_hadm

subject_id   hadm_id  \
1161       16477997  25359811   
1223       14959277  28740655   
1435       18894258  28336555   
1662       14974219  20480369   
1677       12849817  27891695   
1787       11508679  25781770   
1840       18151621  25727457   
2520       12711129  26466704   
2622       18982551  28769191   
2716       13427502  20353250   
2747       18751817  29991380   
2785       12024697  20302177   
2857       11040919  27782536   
2961       19545081  27086801   
3423       15784822  28147695   
3671       15209971  20858433   
3723       13069434  22980160   
3795       13976907  28205831   
3897       12773842  29190436   
4095       13778272  29846444   
4142       16730752  21584004   
4179       18567594  29828717   
4216       19945174  29312839   
4228       16310578  26142163   
4248       19599475  21996765   
4386       16553130  22715553   
4417       17447473  22293942   
4514       13031420  21095600   
4539       15014033  25320448   
4590       17600934  26823201   
...             ...       ...   
5273825    10708393  26497591   
5274100    18893669  27660170   
5274218    17773589  24213883   
5274365    11817632  28793008   
5274852    18454049  29887383   
5274909    16848733  25023099   
5274999    19550378  27016663   
5275662    13169119  28118117   
5275906    12704304  20421160   
5276010    13356023  22685718   
5276608    12841732  28637455   
5276874    18560132  20165317   
5276977    18877772  25562669   
5277438    10582415  28717822   
5277554    18890224  21353088   
5277679    14109642  29257732   
5278128    18286387  29316440   
5278249    18520455  21012517   
5278572    19105785  23698764   
5278643    10412483  24483972   
5278743    11278868  27727945   
5278862    16393972  23399256   
5278898    12853711  25849371   
5278912    13246267  23939582   
5279112    18115365  29229210   
5279160    16799562  23401334   
5279392    15137260  28409503   
5279757    12450853  21128117   
5280049    16050305  29914393   
5280251    17533213  27704968   

                                                long_title icd_code  \
1161               acute on chronic systolic heart failure    42823   
1223                 congestive heart failure, unspecified     4280   
1435                 congestive heart failure, unspecified     4280   
1662                 congestive heart failure, unspecified     4280   
1677                 congestive heart failure, unspecified     4280   
1787                 congestive heart failure, unspecified     4280   
1840                 congestive heart failure, unspecified     4280   
2520              acute on chronic diastolic heart failure    42833   
2622                 congestive heart failure, unspecified     4280   
2716                 congestive heart failure, unspecified     4280   
2747                 congestive heart failure, unspecified     4280   
2785               acute on chronic systolic heart failure    42823   
2857                 congestive heart failure, unspecified     4280   
2961              acute on chronic diastolic heart failure    42833   
3423                 congestive heart failure, unspecified     4280   
3671                 congestive heart failure, unspecified     4280   
3723                 congestive heart failure, unspecified     4280   
3795                 congestive heart failure, unspecified     4280   
3897                 congestive heart failure, unspecified     4280   
4095                 congestive heart failure, unspecified     4280   
4142               acute on chronic systolic heart failure    42823   
4179                 congestive heart failure, unspecified     4280   
4216     acute on chronic combined systolic and diastol...    42843   
4228                 congestive heart failure, unspecified     4280   
4248                 congestive heart failure, unspecified     4280   
4386                 congestive heart failure, unspecified     4280   
4417                 congestive heart failure

## Part III: DRG codes

In [14]:
drgcodes = pd.read_csv(f'{mimiciv_dir}/drgcodes.csv', low_memory=False)

In [15]:
# merge tables
df_ids_icds = pd.merge(df_event, df_code, on = ['icd_code','icd_version'], how='left')
df_merged = pd.merge(df_ids_icds, drgcodes, on = ['hadm_id', 'subject_id'], how='left')
df_merged = df_merged.drop(columns=['drg_severity', 'drg_mortality'])

In [16]:
# drop any rows that don't have a drg_code
df_merged = df_merged.dropna(subset=['drg_code'], how='any')

In [17]:
# select rows where DRG code contains "hf" or "heart failure"
df_sub = df_merged[df_merged['description'].str.contains('hf|heart failure', case=False)]
# there are some drg codes that specify "w/o hf" or "w/o heart failure" - don't include those rows
df_sub = df_sub[~df_sub['description'].str.contains('W/O AMI/HF/SHOCK|W/O AMI, HEART FAILURE OR SHOCK', case=False)]

In [18]:
# remove the hadm_ids that have a long title with heart failure in them, since the ICD codes already account for them
no_hf_icd = []
for hadm_id in df_sub.hadm_id.unique():
    df_hadm_id = df_sub[df_sub['hadm_id'] == hadm_id]
    if len(df_hadm_id[df_hadm_id['long_title'].str.contains('heart failure', case=False)]) == 0:
        no_hf_icd.append(hadm_id)
df_no_hf_icd = df_sub[df_sub['hadm_id'].isin(no_hf_icd)]

In [19]:
# only keep rows where ICD codes include:
#" cardiogenic shock", "left ventricular failure, unspecified", "shock, unspecified", or pleural effusion or fluid overload
df_no_hf_icd = df_no_hf_icd[df_no_hf_icd['long_title'].str.contains('cardiogenic shock|left ventricular failure, unspecified|shock, unspecified|Pleural effusion, not elsewhere classified|Fluid overload, unspecified', case=False)]
df_no_hf_icd_hadm = df_no_hf_icd[['subject_id','hadm_id','long_title', 'icd_code','icd_version', 'description']].drop_duplicates(subset=['hadm_id'])

## Part IV: IV Diuretics

In [20]:
chf_hadm = df_f7_hadm.hadm_id.append(df_no_hf_icd_hadm.hadm_id).unique() # 51322 hadm
len(chf_hadm)

51322

In [21]:
prescriptions = pd.read_csv(f'{mimiciv_dir}/prescriptions.csv', low_memory=False)

In [22]:
df_diuretics = pd.DataFrame({'furosemide': (prescriptions['drug'].str.contains('furosemide', case=False, na=False) | prescriptions['drug'].str.contains('lasix', case=False, na=False)) & ~prescriptions['drug'].str.contains('Desensitization', case=False, na=False), 
                   'bumetanide': (prescriptions['drug'].str.contains('bumetanide', case=False, na=False) | prescriptions['drug'].str.contains('bumex', case=False, na=False)) & ~prescriptions['drug'].str.contains('Desensitization', case=False, na=False), 
                   'chlorothiazide': (prescriptions['drug'].str.contains('chlorothiazide', case=False, na=False) | prescriptions['drug'].str.contains('diuril', case=False, na=False)) & ~prescriptions['drug'].str.contains('Desensitization', case=False, na=False) & ~prescriptions['drug'].str.contains('hydrochlorothiazide', case=False, na=False)})

In [23]:
# prescriptions for diuretics
given_diuretic = df_diuretics.any(axis=1)

In [24]:
# prescriptions with hadm_id associated w heart failure ICD
in_icd = np.isin(prescriptions.hadm_id, chf_hadm)

In [25]:
# prescriptions given via IV
route_iv = prescriptions.route.str.contains('IV', case=False, na=False)

In [26]:
# final cohort with all prescription fields
cohort = prescriptions[given_diuretic & in_icd & route_iv]

In [27]:
# unique drug names to confirm no unwanted drugs are included
cohort.drug.unique()

array(['Furosemide', 'Chlorothiazide Sodium', 'Furosemide-Heart Failure',
       'Chlorothiazide', 'Bumetanide', 'Furosemide (Latex Free)', 'LaSIX',
       'bumex'], dtype=object)

In [28]:
cohort.hadm_id.unique().shape[0] #25691 unique HADM_IDs

25691

In [41]:
cohort.subject_id.unique().shape[0] #14757 unique SUBJECT_IDs

14757

In [30]:
cohort_hadm = cohort.hadm_id.unique()

## Part V: Removing Dialysis Patients 

In [31]:
dialysis_codes = ["Z992", "N186", "5856", "V4511"]

In [32]:
df_event["chronic_dialysis"] = df_event["icd_code"].str.strip().isin(dialysis_codes)

In [33]:
df_event

subject_id   hadm_id  seq_num icd_code  icd_version  chronic_dialysis
0          15734973  20475282        3     2825            9             False
1          15734973  20475282        2    V0251            9             False
2          15734973  20475282        5     V270            9             False
3          15734973  20475282        1    64891            9             False
4          15734973  20475282        4    66481            9             False
5          11442057  21518990        1    65971            9             False
6          11442057  21518990        2    64231            9             False
7          11442057  21518990        6     V270            9             False
8          11442057  21518990        3    64511            9             False
9          11442057  21518990        5    65961            9             False
10         11442057  21518990        4    65911            9             False
11         10072949  20817034        4     7840            9             False
12         10072949  20817034        2    65221            9             False
13         10072949  20817034        1    64231            9             False
14         10072949  20817034        5     V270            9             False
15         10072949  20817034        3    64681            9             False
16         13538405  24822466        5    V0251            9             False
17         13538405  24822466        2     V270            9             False
18         13538405  24822466        3    65611            9             False
19         13538405  24822466        1    66411            9             False
20         13538405  24822466        4    64891            9             False
21         10287061  27485182        6    66411            9             False
22         10287061  27485182        5    V0251            9             False
23         10287061  27485182        2    64231            9             False
24         10287061  27485182        3     V270            9             False
25         10287061  27485182        1    64511            9             False
26         10287061  27485182        4    64891            9             False
27         10287061  27485182        7    75617            9             False
28         16016170  21252749        3     V270            9             False
29         16016170  21252749        2    66331            9             False
...             ...       ...      ...      ...          ...               ...
5280321    12225682  29240982        5     F329           10             False
5280322    12225682  29240982        2   F10129           10             False
5280323    12225682  29240982        3   F14929           10             False
5280324    12225682  29240982        4   F15929           10             False
5280325    12225682  29240982        1     F419           10             False
5280326    12225682  29240982        6     Z915           10             False
5280327    12225682  29240982        8    Z9114           10             False
5280328    15119129  28721266        2     Y836           10             False
5280329    15119129  28721266        4     Y929           10             False
5280330    15119129  28721266        3      I10           10             False
5280331    15119129  28721266        1  T8132XA           10             False
5280332    19118815  29566765        5      I10           10             False
5280333    19118815  29566765        6   Z87891           10             False
5280334    19118815  29566765        2     C329           10             False
5280335    19118815  29566765        4   Z13220           10             False
5280336    19118815  29566765        1  M84422A           10             False
5280337    19118815  29566765        3   G40909           10             False
5280338    11735767  27197374        1     P599           10             False
5280339    13747041  25594844       12    Z8619           1

## Part VI: Final Cohort

In [34]:
final_cohort = df_event.drop(columns=['seq_num', 'icd_code', 'icd_version'])
final_cohort.head()

subject_id   hadm_id  chronic_dialysis
0    15734973  20475282             False
1    15734973  20475282             False
2    15734973  20475282             False
3    15734973  20475282             False
4    15734973  20475282             False

In [35]:
final_cohort['CHF exacerbation'] = final_cohort['hadm_id'].isin(cohort_hadm)

In [36]:
final_cohort = final_cohort.drop_duplicates()

In [37]:
final_cohort.head()

subject_id   hadm_id  chronic_dialysis  CHF exacerbation
0     15734973  20475282             False             False
5     11442057  21518990             False             False
11    10072949  20817034             False             False
16    13538405  24822466             False             False
21    10287061  27485182             False             False

In [38]:
len(final_cohort)

536165

In [39]:
final_cohort.to_csv('final_cohort.csv')

In [56]:
sum(final_cohort['chronic_dialysis']==False)

521111

In [64]:
final_cohort_selected = final_cohort[(final_cohort['chronic_dialysis']==False) & (final_cohort['CHF exacerbation']==True)]
final_cohort_selected.head()
final_cohort_selected.to_csv('final_cohort_selected.csv')

In [65]:
len(final_cohort_selected)

25691